# Maps Kevin's data onto reactome pathways

In [1]:
%load_ext autoreload
%autoreload 2

In [24]:
import os
import glob

import pylab as plt
import matplotlib
from IPython.display import display, HTML

import numpy as np
from scipy.sparse import lil_matrix
import pandas as pd
from pandas import HDFStore

import seaborn as sns
from IPython.display import display

from reactome import ensembl_to_uniprot, uniprot_to_reaction, compound_to_reaction, \
    reaction_to_metabolite_pathway, get_reaction_ids, get_reaction_df, get_reaction_entities, \
    produce_kegg_dict
    
from collections import defaultdict

from ipywidgets import FloatProgress

%matplotlib inline

## Load data

In [3]:
basedir = '/Users/joewandy/Dropbox/Analysis/omics_integration'

In [4]:
time = 7
parasite = 'INFEC'
treatment = 'Unsorted'

filename = os.path.join(basedir, 'data/data_%d_%s_%s.h5' % (time, parasite, treatment))
hdf = HDFStore(filename, complevel=9, complib='bzip2')

pvt = hdf['peak_vs_transcript'].replace([np.inf, -np.inf], np.nan).fillna(0)
# pvp = hdf['peak_vs_peak']
# tvt = hdf['transcript_vs_transcript']

hdf.close()

In [5]:
pvt.columns

Index([u'ENSMUSG00000000001', u'ENSMUSG00000000028', u'ENSMUSG00000000031',
       u'ENSMUSG00000000049', u'ENSMUSG00000000056', u'ENSMUSG00000000058',
       u'ENSMUSG00000000078', u'ENSMUSG00000000085', u'ENSMUSG00000000088',
       u'ENSMUSG00000000093',
       ...
       u'ENSMUSG00000110391', u'ENSMUSG00000110393', u'ENSMUSG00000110397',
       u'ENSMUSG00000110399', u'ENSMUSG00000110404', u'ENSMUSG00000110405',
       u'ENSMUSG00000110410', u'ENSMUSG00000110414', u'ENSMUSG00000110419',
       u'ENSMUSG00000110424'],
      dtype='object', length=21794)

In [6]:
species = 'Mus musculus'

## Maps ENSEMBL IDs to UniProt IDs

In [7]:
ens_id = pvt.columns[0]
print ens_id

ENSMUSG00000000001


In [8]:
print len(pvt.columns)
ens_ids = pvt.columns.values.tolist()

21794


In [9]:
transcript_mapping = ensembl_to_uniprot(ens_ids, species, show_progress_bar=True)

FloatProgress(value=0.0, max=21794.0)

In [10]:
ensembl_ids = transcript_mapping.keys()[0:10]
for ensembl_id in ensembl_ids:
    print ensembl_id, transcript_mapping[ensembl_id]

ENSMUSG00000002010 [u'P70404']
ENSMUSG00000002015 [u'Q61335']
ENSMUSG00000026020 [u'Q6DFW4']
ENSMUSG00000074886 [u'O70293']
ENSMUSG00000006005 [u'F6ZDS4']
ENSMUSG00000003464 [u'Q8VCI5']
ENSMUSG00000004895 [u'Q9EQC8']
ENSMUSG00000046027 [u'Q9EPQ7']
ENSMUSG00000046756 [u'Q80X85']
ENSMUSG00000004892 [u'Q61361']


In [11]:
total_mapped = len(transcript_mapping)
total_genes = len(ens_ids)
certain = []
for ensembl_id in transcript_mapping:
    if len(transcript_mapping[ensembl_id]) == 1:
        certain.append(transcript_mapping[ensembl_id][0])

uncertain = total_mapped - len(certain)
print len(certain), uncertain, total_mapped, total_genes

5987 613 6600 21794


## Map UniProt IDs to Reactions

In [12]:
protein_mapping = uniprot_to_reaction(certain, species, show_progress_bar=True)

FloatProgress(value=0.0, max=5987.0)

In [13]:
for uniprot_id in certain[0:3]:
    print uniprot_id, protein_mapping[uniprot_id]
    print

P70404 [{'reaction_id': u'R-MMU-70967', 'reaction_name': u'isocitrate + NAD+ => alpha-ketoglutarate + CO2 + NADH + H+ [IDH3]'}]

Q61335 [{'reaction_id': u'R-MMU-351894', 'reaction_name': u'Caspase mediated cleavage of BAP31'}]

Q6DFW4 [{'reaction_id': u'R-MMU-4570467', 'reaction_name': u'SUMOylation of NOP58 with SUMO1'}, {'reaction_id': u'R-MMU-6791222', 'reaction_name': u'21S pre-rRNA is nucleolytically processed at site E (site2a) to yield 18SE pre-rRNA'}, {'reaction_id': u'R-MMU-6791227', 'reaction_name': u"47S pre-rRNA is nucleolytically processed at A' (01,A1), site A0, and site 02 (site 6)  to yield 45S pre-rRNA"}]



## Map KEGG Compound IDs to Reactions

Load a dataframe containing the KEGG IDs of identified compounds only

In [14]:
peak_df = pd.read_csv(os.path.join(basedir, 'data/my_analysis_peaks.csv'), index_col=0)

In [15]:
peak_df.shape

(109, 11)

In [16]:
peak_df.head()

,sec_id,mass,rt,polarity,c_id,formula,adduct,rc_id,compound,db,identifier
pid,,,,,,,,,,,
741583,1,147.076381,905.000626,positive,2825862,C5H10N2O3,M+H,3453657,L-Glutamine,kegg,C00064
741598,16,162.076139,891.966177,positive,2826034,C6H11NO4,M+H,3453882,L-2-Aminoadipate,kegg,C00956
741599,17,116.070596,771.412376,positive,2826041,C5H9NO2,M+H,3453891,L-Proline,kegg,C00148
741611,29,132.101919,677.261473,positive,2826103,C6H13NO2,M+H,3453976,L-Leucine,kegg,C00123
741680,98,132.101919,646.385592,positive,2826221,C6H13NO2,M+H,3454135,L-Leucine,kegg,C00123


In [17]:
compound_ids = sorted(list(set(peak_df['identifier'].values.tolist())))

In [18]:
compound_mapping = compound_to_reaction(compound_ids, species, show_progress_bar=True)

FloatProgress(value=0.0, max=70.0)

In [19]:
print compound_mapping.keys()[0:10]

[u'C00025', u'C00328', u'C00149', u'C00020', u'C00009', u'C00147', u'C00013', u'C00026', u'C00864', u'C00099']


## Map Reactions to Pathways

In [20]:
protein_reactions = list(set(get_reaction_ids(protein_mapping)))
compound_reactions = list(set(get_reaction_ids(compound_mapping)))
combined_reactions = list(set(protein_reactions + compound_reactions))

print len(protein_reactions)
print len(compound_reactions)
print len(combined_reactions)

7096
870
7227


In [21]:
print combined_reactions[0:10]

[u'R-MMU-9014672', u'R-MMU-2130336', u'R-MMU-5694018', u'R-MMU-6813740', u'R-MMU-9014678', u'R-MMU-204485', u'R-MMU-202626', u'R-MMU-1254386', u'R-MMU-5693561', u'R-MMU-8939706']


This is a map of reactions to pathways that are under the top-level biochemical ('Metabolism') pathways

In [28]:
pathway_mapping = reaction_to_metabolite_pathway(combined_reactions, species, show_progress_bar=True, last_pathway=True)
print len(pathway_mapping)
print pathway_mapping['R-MMU-2395768']

FloatProgress(value=0.0, max=7227.0)

1386
[{'pathway_id': u'R-MMU-975634', 'pathway_name': u'Retinoid metabolism and transport'}]


In [ ]:
kegg_location = os.path.join(basedir, 'data/kegg/kegg.xml')
kegg_dict = produce_kegg_dict(kegg_location, 'id')

In [ ]:
reaction_df = get_reaction_df(transcript_mapping, protein_mapping, compound_mapping, pathway_mapping, species)
reaction_df.sort_values(by=['compound_coverage', 'protein_coverage'], inplace=True, ascending=False)
print reaction_df.shape

In [ ]:
reaction_df

In [ ]:
reaction_df.to_csv('reaction_df.tsv', sep='\t', index=False)

How many unique pathways?

In [ ]:
pathway_names = []
for names in reaction_df['pathway_names'].values:
    ps = names.split(':')
    pathway_names.extend(ps)
unique_names = sorted(list(set(pathway_names)))

In [ ]:
pathway_df = pd.DataFrame(unique_names, columns=['pathway_name'])
pathway_df.to_csv('pathway_df.csv', index=False)
print pathway_df.shape

In [ ]:
pathway_df